# Solving the S-NDR Equations in 1D

The equations are,

$$ \partial_t C_i = D_i \partial_x^2 C_i $$

for $i=\text{Cu},\;\text{sup}$ and

$$ \partial_t \theta = k^+ c_{\text{sup}} \left( 1 - \theta \right) - k^- \theta v $$

where

$$ v = j_0 \left( 1 - \theta \right) + j_1 \theta $$

The current is given by,

$$ j_k = j_k^{\alpha} \left[ \exp \left( \alpha \bar{\eta} \right) - \exp \left( -\left(1 - \alpha \right) \bar{\eta} \right) \right] \frac{ c_{\text{Cu}} } { c_{\text{Cu}}^{\infty} }$$

for $k=0, 1$ and $\bar{\eta} = \eta\frac{n F}{R T}$.

with

$$\eta = V_0 + V_{ramp} t$$

when $t < \frac{t_f}{2}$ and

$$\eta = V_0 + V_{ramp} \left(t_f - t\right)$$

when $t > \frac{t_f}{2}$ and

## Initial Conditions

$\theta = 0$, $c_{\text{Cu}} = 0$ and $c_{\text{suppressor}} = 0$

## Boundary Conditions

$c_{\text{Cu}} = c_{\text{Cu}}^{\infty}$ at the boundary layer

$c_{\text{sup}} = c_{\text{sup}}^{\infty}$ at the boundary layer

$-D_{\text{Cu}} \partial_x c_{\text{Cu}} = \frac{v}{\Omega}$ at the electrode

$-D_{\text{sup}} \partial_x c_{\text{sup}} = c_{\text{sup}} k^+ \Gamma \left( 1 - \theta \right)$ at the electrode

## Parameters



|           Symbol           |                 Value (a) |
|:--------------------------:|--------------------------:|
|       $D_\text{supp}$      |   $9.2\;\times\;10^{-11}$ |
|       $D_\text{Cu}$        | $2.65\;\times\;10^{-10}$  |
| $c_{\text{Cu}}^{\infty}$   |                   240     |
| $c_{\text{sup}}^{\infty}$  | 0.01                      |
|       $\delta$             |   $1\;\times\;10^{-6}$    |
|  $ \Gamma  $                 |   $2.5\;\times\;10^{-7}$  |
| $k^+$                      |          2300             |
| $k^-$                      |   $3.79\;\times\;10^{7}$  |
| $j_0^{\alpha}$             |          20               |
| $j_1^{\alpha}$             |   $1\;\times\;10^{-3}$    |
| $\alpha$                   |   0.4                     |
| $n$                        |   2                       |
| $\Omega$                   |   $7.2\;\times\;10^{-6}$  |
| $F$                        |  96485.332                |
| $V_0$ (initial potential)  | 0                         |
| $V_s$ (switch)             | -0.325                    |
| $V_{ramp}$ (ramp rate)        | 0.01                      |
| Final time $t_f$           | 65                        |

In [4]:
import matplotlib as mpl
mpl.rcParams.update({'font.size': 22})
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']
import matplotlib.pyplot as plt
from main import run, get_params
import numpy as np
from ipywidgets import interact_manual, IntSlider, FloatSlider, BoundedFloatText, interact
from toolz.curried import pipe, curry

In [5]:
@curry
def plot_mpl(params, values):
    x = np.array(values['sup'].mesh.x) / params['delta']
    sup = np.array(values['sup']) / params['sup_inf']
    cupric = np.array(values['cupric']) / params['cupric_inf']
    plt.figure(figsize=(11, 8))
    plt.plot(x, sup, label=r'$i=\text{supp}$')
    plt.plot(x, cupric, label=r'$i=\text{cupric}$')
    plt.ylim(-0.02, 1.02)
    plt.xlabel(r'$x / \delta$')
    plt.ylabel(r'$c_i / c^{\infty}_i$')
    plt.legend()
    plt.show()

In [6]:
# NBVAL_IGNORE_OUTPUT

@interact.options(manual=True)
def run_plot(
    dt=FloatSlider(value=0.1, min=0.01, max=1, step=0.01),
    max_steps=IntSlider(min=1, max=100, step=10, value=10),
    max_sweeps=IntSlider(min=1, max=20, step=1, value=5),
    k_plus=BoundedFloatText(value=2300.0, min=1., max=1e+10, step=1),
    k_minus=BoundedFloatText(value=3.79e+7, min=1., max=1e+10, step=1),            
    output=False):
    return pipe(
        dict(
            get_params(),
            dt=dt,
            max_steps=max_steps,
            max_sweeps=max_sweeps,
            k_plus=k_plus,
            k_minus=k_minus,
            output=output
        ),
        lambda x: plot_mpl(x, run(x))
    )



interactive(children=(FloatSlider(value=0.1, description='dt', max=1.0, min=0.01, step=0.01), IntSlider(value=10, description='max_steps', min=1, step=10), IntSlider(value=5, description='max_sweeps', max=20, min=1), BoundedFloatText(value=2300.0, description='k_plus', max=10000000000.0, min=1.0, step=1.0), BoundedFloatText(value=37900000.0, description='k_minus', max=10000000000.0, min=1.0, step=1.0), Checkbox(value=False, description='output'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))